In [1]:
# Import necessary packages and libraries
import pandas as pd
import json
from PIL import Image, ImageDraw, ImageFont
from PIL.ImageOps import scale
import math
from itertools import groupby
import numpy as np
from statistics import mean

In [2]:
class BackgroundTemplate:
    '''
    Description: Background Template helps to define box methods to create the graphical abstract template 
    which will house all the necessary information needed for the graphical abstract visualizer
    '''
    def __init__(self, image_size, background_color='white'):
        self.image = Image.new('RGB', image_size, color=background_color)
        self.draw = ImageDraw.Draw(self.image)
        self.boxes = []

    def add_box(self, top_left, bottom_right, outline_color='black', fill_color=None, text=None, text_color='black'):
        box = {'top_left': top_left, 'bottom_right': bottom_right, 'outline_color': outline_color, 'fill_color': fill_color, 'text': text, 'text_color': text_color}
        self.boxes.append(box)

    def draw_boxes(self):
        for box in self.boxes:
            self.draw.rectangle([box['top_left'], box['bottom_right']], outline=box['outline_color'], fill=box['fill_color'])
            if box['text']:
                self._draw_centered_text(box['text'], box['top_left'], box['bottom_right'], box['text_color'])

    def _draw_centered_text(self, text, top_left, bottom_right, text_color, font = None):
        if font == None:
            font_path = "./fonts/Aptos-Bold.ttf"
            font_size = 26
            font = ImageFont.truetype(font_path, font_size)
        bbox = self.draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        box_width = bottom_right[0] - top_left[0]
        box_height = bottom_right[1] - top_left[1]
        x = top_left[0] + (box_width - text_width) / 2
        y = top_left[1] + (box_height - text_height) / 2
        self.draw.text((x, y), text, fill=text_color, font=font)


    def get_image(self):
        return self.image

    def paste(self, icon, icon_position, icon_type):
        self.image.paste(icon, icon_position, icon_type)

    def add_enclosed_boxes(self, enclosing_box_top_left, enclosing_box_bottom_right, num_inner_boxes, spacing, colors, texts):
        # Define the enclosing box
        self.add_box(enclosing_box_top_left, enclosing_box_bottom_right, outline_color='black', fill_color=None)
        
        # Calculate inner box dimensions and spacing
        enclosing_width = enclosing_box_bottom_right[0] - enclosing_box_top_left[0]
        enclosing_height = enclosing_box_bottom_right[1] - enclosing_box_top_left[1]
        box_width = enclosing_width - 2 * spacing
        box_height = (enclosing_height - (num_inner_boxes + 1) * spacing) // num_inner_boxes
        
        # Add inner boxes
        for i in range(num_inner_boxes):
            top_left = (enclosing_box_top_left[0] + spacing, enclosing_box_top_left[1] + spacing + i * (box_height + spacing))
            bottom_right = (top_left[0] + box_width, top_left[1] + box_height)
            self.add_box(top_left, bottom_right, outline_color='black', fill_color=colors[i % len(colors)], text=texts[i], text_color='white')

In [3]:
def duration_to_arrow_length(template, group_data, x, y, y_increment, min_length=40, max_length=670):
            
    
    
    recovery_times = group_data.groupby(['Rad type','Expt group', 'group-subgroup', 'Dose', 'Sex', 'Countermeasure'])['Recovery (d)'].mean().reset_index()
    recovery_times = recovery_times.sort_values(by='Sex')
    print(recovery_times)
    
    min_weeks = 1
    max_weeks = math.ceil(float(max(recovery_times['Recovery (d)'])) / 7.0)
    for days in recovery_times['Recovery (d)']:
        week = math.ceil(float(days) / 7.0)
        length = ((week - min_weeks) / (max_weeks - min_weeks)) * (max_length - min_length) + min_length
        arrow_length = max(min_length, min(length, max_length))
        arrow_start = (x , y)
        arrow_end = (arrow_start[0] + arrow_length, arrow_start[1])
        template.draw.line([arrow_start, arrow_end], fill='black', width=1)
        arrowhead = [(arrow_end[0], arrow_end[1]), (arrow_end[0] - 2, arrow_end[1] - 2),
                (arrow_end[0] - 2, arrow_end[1] + 2)]
        template.draw.polygon(arrowhead, fill='black')
        y += y_increment
         
    return "Drawing Arrows Complete"

In [4]:
def add_labels(template, group_data, initial_pos, initial_age, pos_y):

    '''
    Description: Labels the age of mice during the treatment procedure from initial age to 
    final termination. This function dynamically scales the number of weeks in or out based on the 
    largest termination date present in the dataset. 
    '''
    scale_value = 2
    
    font_path = "./fonts/Aptos-Bold.ttf"
    font_size = 10 * scale_value
    font = ImageFont.truetype(font_path, font_size)
    
    weeks_to_display = []       
    weeks_to_display += [int(math.ceil(float(group) / 7.0)) for group in group_data['Recovery (d)']]
    max_duration = max(weeks_to_display)

    pos_x = initial_pos
    prev = 0 

    if max_duration <= 20: 
        week_labels = range(initial_age, (initial_age+max_duration), 1)
    elif max_duration <= 50:
        week_labels = range(initial_age, (initial_age+max_duration), 2) 
    elif max_duration <= 200:
        week_labels = range(initial_age, (initial_age+max_duration), 5)
    else:
        week_labels = range(initial_age, (initial_age+max_duration), 10)
        

    template._draw_centered_text("Age (Wks)", (pos_x - 70 * scale_value, pos_y), (pos_x + 20 * scale_value, pos_y), text_color = 'black', font = font)
    template.add_box((pos_x, pos_y), (pos_x + 0.0001 * scale_value, pos_y + 570 * scale_value), outline_color= '#ebedf0', fill_color= None, text=None, text_color='black')
    
    for week in week_labels:
        pos_x += ((1300 * scale_value - initial_pos) / max_duration) * (week - prev - initial_age)
        template._draw_centered_text(f"{week}", (pos_x, pos_y), (pos_x + 20, pos_y), text_color = 'black', font = font)
        template.add_box((pos_x, pos_y), (pos_x + 0.0001 * scale_value, pos_y + 570 * scale_value), outline_color= '#ebedf0', fill_color= None , text=None, text_color='black')
        prev = week - initial_age
    
    # pos_x += ((1300 - initial_pos) / max_duration) * (max_duration - prev)
    # template._draw_centered_text(f"{max_duration+initial_age}", (pos_x, pos_y), (pos_x + 20, pos_y), text_color = 'black', font = font)
    # template.add_box((pos_x, pos_y), (pos_x + 0.0001, pos_y + 570), outline_color='gray', fill_color=None, text=None, text_color='black')
    # prev = week - initial_age

In [5]:
 def draw_arrow(template, recovery_time, subgroup_value, x, y, min_length=40, max_length=670):
    scale_value = 2
    font_path = "./fonts/Aptos-Bold.ttf"
    font_size = 10 * scale_value
    font = ImageFont.truetype(font_path, font_size)
     
    recovery_times = group_data.groupby(['Rad type','Expt group', 'group-subgroup', 'Dose (Gy)', 'Sex', 'Countermeasure'])['Recovery (d)'].mean().reset_index()
    recovery_times = recovery_times.sort_values(by=['Expt group','Countermeasure', 'Sex'])
     
    min_weeks = 1
    max_weeks = math.ceil(float(max(recovery_times['Recovery (d)'])) / 7.0)
    week = math.ceil(float(recovery_time) / 7.0)
    length = ((week - min_weeks) / (max_weeks - min_weeks)) * (max_length - min_length) + min_length
    arrow_length = max(min_length, min(length, max_length))
    arrow_start = (x , y)
    arrow_end = (arrow_start[0] + arrow_length, y + 2 * scale_value)
    template.add_box(arrow_start, arrow_end, outline_color= None, fill_color='black')
    arrowhead = [(arrow_end[0], arrow_end[1]), (arrow_end[0] - 2 * scale_value, arrow_end[1] - 2 * scale_value),
            (arrow_end[0] - 2 * scale_value, arrow_end[1] + 2 * scale_value)]
    template.draw.polygon(arrowhead, fill='black')

    number_of_subjects = group_data.groupby('group-subgroup').count()

    for i, value in number_of_subjects.iterrows():
        if i == subgroup_value:
            print(value.iloc(0)[0])
            template._draw_centered_text(f"{value.iloc(0)[0]} Subjects", (arrow_end[0] + 20 * scale_value, arrow_end[1]), (arrow_end[0] + 50 * scale_value, arrow_end[1]), text_color = 'black', font = font)
         
    return "Arrow Drawn"

In [6]:
scale_value = 2
male_icon_path = 'male.png'
female_icon_path = 'female.png'
male_icon = Image.open(male_icon_path)
female_icon = Image.open(female_icon_path)
new_icon_size = (50 * scale_value, 50 * scale_value)
male_icon_resized = male_icon.resize(new_icon_size)
female_icon_resized = female_icon.resize(new_icon_size)

In [7]:
scale_value = 2
pos_icon_path = 'pos.png'
neg_icon_path = 'neg.png'
pos_icon = Image.open(pos_icon_path)
pos_icon = pos_icon.convert("P")
neg_icon = Image.open(neg_icon_path)
new_icon_size = (60 * scale_value, 60 * scale_value)
pos_icon_resized = pos_icon.resize(new_icon_size)
neg_icon_resized = neg_icon.resize(new_icon_size)

In [8]:
dos_icon_path = 'radiation.png'
dos_icon = Image.open(dos_icon_path)

In [9]:
def create_experiment_structure(template, group_data, sheet_name):

    scale_value = 2

    font_path = "./fonts/Aptos-Bold.ttf"
    font_small = 12 * scale_value
    font_large = 14 * scale_value
    font_small = ImageFont.truetype(font_path, font_small)
    font_large = ImageFont.truetype(font_path, font_large)

    # # Create the dataframes for labelling the template
    # df = pd.read_excel('Modified Tissue Archive for Graphical Abstract Generation.xlsx')
    # df2 =  pd.read_excel('Experiment Info.xlsx')

    # # Modify the dataframes to have the right headers
    # group_data = df[1:]
    # group_data.columns = df.iloc[0]

    # Sort and Group the Dataframe according to the Radiation, Experimental Group / Subgroup, Gender and Countermeasure
    recovery_times = group_data.groupby(['Rad type','Expt group', 'group-subgroup', 'Dose (Gy)', 'Sex', 'Countermeasure'])['Recovery (d)'].mean().reset_index()
    recovery_times = recovery_times.sort_values(by=['Expt group','Countermeasure', 'Sex'])
    recovery_times

    rad_types = list(recovery_times['Rad type'].unique())
    experimental_groups = list(recovery_times['Dose (Gy)'].unique())
    counter_groups = list(recovery_times['Countermeasure'].unique())
    gender_groups = list(recovery_times['Sex'].unique())

    x = 630 * scale_value
    y = 100 * scale_value
    arrow_y = []
    gender_y = []
    counter_y = []
    group_y = []
    rad_y = []

    for rad_type in rad_types:
        # print(rad_type)
        for group in experimental_groups:
            filtered_df = recovery_times.loc[recovery_times['Dose (Gy)'] == group]
            if filtered_df.empty:
                continue
            else: 
                # print(filtered_df)
                for counter in counter_groups:
                    counter_df = filtered_df.loc[filtered_df['Countermeasure'] == counter]
                    if counter_df.empty:
                        continue
                    else: 
                        # print(counter_df)
                        for gender in gender_groups: 
                            gender_df = counter_df.loc[counter_df['Sex'] == gender]
                            if gender_df.empty:
                                continue
                            else: 
                                # print(gender_df)
                                for index, row in gender_df.iterrows():
                                    draw_arrow(template, row['Recovery (d)'], row['group-subgroup'], x, y, min_length=40 * scale_value, max_length=670 * scale_value)
                                    arrow_y.append(y)
                                    y += 10 * scale_value
                            min_g = min(arrow_y)
                            max_g = max(arrow_y)
                            counter_value = int(((min_g + max_g)/2)) - 20
                            if gender == 'M':
                                gender = 'Male'
                                template.paste(male_icon_resized, (580 * scale_value, counter_value), male_icon_resized.convert('RGBA'))
                            else:
                                gender = 'Female'
                                template.paste(female_icon_resized, (580 * scale_value, counter_value), female_icon_resized.convert('RGBA'))
                            #template._draw_centered_text(f"Gender: {gender}", (540, min_g), (540 + 20, max_g), text_color = 'black', font = font_small)
                            counter_y.append(counter_value)
                            arrow_y = []
                            y += 20 * scale_value
                    min_c = min(counter_y)
                    max_c = max(counter_y)
                    gender_value = int(((min_c + max_c)/2))
                    if counter == 'Y':
                            counter = 'Yes'
                            template.paste(pos_icon_resized, (500 * scale_value, gender_value), pos_icon_resized.convert('RGBA'))
                    else:
                            gender = 'No'
                            template.paste(neg_icon_resized, (500 * scale_value, gender_value), neg_icon_resized.convert('RGBA'))
                    #template._draw_centered_text(f"Countermeasure: {counter}", (500, min_c), (500 + 20, max_c), text_color = 'black', font = font_small)
                    group_y.append(gender_value)
                    counter_y = []
                    y += 40 * scale_value
                    # print(y)
            min_gr = min(group_y)
            max_gr = max(group_y)
            rad_value = int(((min_gr + max_gr)/2))
            if group > 0 and group <= 2: 
                icon_x = (int(group * 30) + 30) * scale_value
                new_icon_size = (icon_x + 5 * scale_value, icon_x)
                dos_icon_resized = dos_icon.resize(new_icon_size)
                template.paste(dos_icon_resized, ((405 - int((group * 30)/2)) * scale_value, rad_value) , dos_icon_resized.convert('RGBA'))
                template._draw_centered_text(f"{group} cGy {sheet_name}", (410 * scale_value, min_gr - 40 * scale_value), ((410 + 20) * scale_value, max_gr + 10 * scale_value), text_color = 'black', font = font_small)
            elif group > 2:
                group  = group/100
                icon_x = (int(group * 30) + 30) * scale_value
                new_icon_size = (icon_x + 5 * scale_value, icon_x)
                dos_icon_resized = dos_icon.resize(new_icon_size)
                template.paste(dos_icon_resized, ((405 - int((group * 30)/2)) * scale_value, rad_value) , dos_icon_resized.convert('RGBA'))
                template._draw_centered_text(f"{group} cGy {sheet_name}", (410 * scale_value, min_gr - 40 * scale_value), ((410 + 20) * scale_value, max_gr + 10 * scale_value), text_color = 'black', font = font_small) 
            else: 
                template._draw_centered_text(f"0 cGy Mock-IR Control", (450 * scale_value, min_gr ), ((450 + 20) * scale_value, max_gr), text_color = 'black', font = font_large)
            rad_y.append(mean([min_gr, max_gr]))
            group_y = []
            y += 80 * scale_value
        min_r = min(rad_y)
        max_r = max(rad_y)
        template._draw_centered_text("Treatment", (510 * scale_value, 70 * scale_value), ((510 + 20) * scale_value, 70 * scale_value), text_color = 'black', font = font_large)
        template._draw_centered_text("(Nicotinamide Riboside)", (510 * scale_value, 90 * scale_value), ((510 + 20) * scale_value, 90 * scale_value), text_color = 'black', font = font_small)
        #template._draw_centered_text(f"{rad_type}", (410, 71), (410 + 20, 71), text_color = 'black', font = font_large)
        y += 160 * scale_value

In [10]:
def generate_image(pi_info, exp_info, group_data, sheet_name):

        scale_value = 2
        
        # Experiment Initialization
        template = BackgroundTemplate((1400 * scale_value, 1400 * scale_value))
        
        font_path = "./fonts/Aptos-Bold.ttf"
        font_size = 26
        font = ImageFont.truetype(font_path, font_size)

        # Define the parameters for the first set of enclosed boxes
        enclosing_box_top_left1 = (50 * scale_value, 50 * scale_value)
        enclosing_box_bottom_right1 = (350 * scale_value, 300 * scale_value)
        num_inner_boxes = 4
        spacing = 20 * scale_value
        colors = ['#649de3', '#649de3', '#649de3', '#649de3']
        texts1 = pi_info

        # Add the first set of enclosed boxes
        template.add_enclosed_boxes(enclosing_box_top_left1, enclosing_box_bottom_right1, num_inner_boxes, spacing, colors, texts1)

        # Define the parameters for the second set of enclosed boxes
        enclosing_box_top_left2 = (50 * scale_value, 320 * scale_value)
        enclosing_box_bottom_right2 = (350 * scale_value, 570 * scale_value)
        num_inner_boxes_2 = 4
        
        texts2 = exp_info

        # Add the second set of enclosed boxes
        template.add_enclosed_boxes(enclosing_box_top_left2, enclosing_box_bottom_right2, num_inner_boxes_2, spacing, colors, texts2)

        # Define the parameters for the third logo box
        logo_box_top_left3 = (50 * scale_value, 590 * scale_value)
        logo_box_bottom_right3 = (350 * scale_value, 650 * scale_value)
        template.add_box(logo_box_top_left3, logo_box_bottom_right3)

        
        
        
        # Define the parameters for the Treatment Group Visualizer
        treatment_box_top_left = (370 * scale_value, 50 * scale_value)
        treatment_box_bottom_right = (1350 * scale_value, 650 * scale_value)
        template.add_box(treatment_box_top_left, treatment_box_bottom_right)

        #Display Weeks
        add_labels(template, group_data, 630 * scale_value, 24, 70 * scale_value)
    
        # duration_to_arrow_length(template, group_data, 630, 120, 10)
        create_experiment_structure(template, group_data, sheet_name)

       

        # Draw all boxes
        template.draw_boxes()
        # Display or save the image
        image_output = template.get_image()
        image_output.show()
        # template.get_image().save(f'{sheet_name}.png')  # Save the image


In [15]:
# df_dict = pd.read_excel('Modified Tissue Archive for Graphical Abstract Generation.xlsx', sheet_name = None)
# df2 =  pd.read_excel('Experiment Info.xlsx')
# df_dict.keys()

df_dict = pd.read_excel('Blakely-Chang_Spreadsheet.xlsx', sheet_name = None)
df2 =  pd.read_excel('Experiment Info.xlsx')
df_dict.keys()

dict_keys(['SI-28 | G380-11', 'Sheet1', 'Modified Tissue Archive'])

In [16]:
for sheet in df_dict.keys():
    print(sheet)
    df = df_dict[sheet]
    group_data = df[1:]
    group_data.columns = df.iloc[0]
    # print(group_data)
    total_info = df2.iloc[0]
    pi_info = total_info[:4]
    exp_info = total_info[4:]
    generate_image(pi_info, exp_info, group_data, sheet)

SI-28 | G380-11
130
121
24
30
74
51


/var/folders/8z/xh7nv8fd54lb36dkhtrtmvsh0000gp/T/ipykernel_54283/881503285.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.add_box(top_left, bottom_right, outline_color='black', fill_color=colors[i % len(colors)], text=texts[i], text_color='white')


Sheet1


IndexError: single positional indexer is out-of-bounds